In [ ]:
from itertools import product
from collections import Counter
import random
import requests
import json
import pandas as pd
import time
# from tqdm import tqdm
from tqdm.auto import tqdm
from getpass import getpass
import uuid

In [ ]:
scenario_data = json.load(open("eng_scenarios_50k.json"))
NUMBER_OF_SCENARIOS = 1500 # len(scenario_data)

## GigaChat Requests

In [ ]:
!wget https://gu-st.ru/content/lending/russian_trusted_root_ca_pem.crt

In [ ]:
AUTHORIZATION_KEY = getpass("Gigachat authorization (base64 encoded Client ID and Client Secret): ")

In [ ]:
url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

payload={"scope": "GIGACHAT_API_PERS"}
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Accept': 'application/json',
  'Authorization': f'Basic {AUTHORIZATION_KEY}',
  'RqUID': str(uuid.uuid4())
}

response = requests.request("POST", url, headers=headers, data=payload, verify="./russian_trusted_root_ca_pem.crt")

print(response.text)

In [ ]:
ACCESS_TOKEN = response.json()["access_token"]
print(ACCESS_TOKEN)

In [ ]:
url = "https://gigachat.devices.sberbank.ru/api/v1/models"

payload={}
headers = {
  'Accept': 'application/json',
  'Authorization': f'Bearer {ACCESS_TOKEN}'
}

response = requests.request("GET", url, headers=headers, data=payload, verify="./russian_trusted_root_ca_pem.crt")

print(response.text)

{"object":"list","data":[{"id":"GigaChat","object":"model","owned_by":"salutedevices"},{"id":"GigaChat-Plus","object":"model","owned_by":"salutedevices"},{"id":"GigaChat-Pro","object":"model","owned_by":"salutedevices"}]}


In [ ]:
def send_gigachat_request(system_content, user_content):
  prompt = {
    "model": "GigaChat",
    "messages": [
      {
        "role": "system",
        "content": system_content
      },
      {
        "role": "user",
        "content": user_content
      }
    ],
    "n": 1,
    "stream": False,
    "update_interval": 0
  }

  url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"
  headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': f'Bearer {ACCESS_TOKEN}'
  }

  response = requests.request("POST", url, headers=headers, data=json.dumps(prompt), verify="./russian_trusted_root_ca_pem.crt")
  # print(response.json())
  return response.json()["choices"][0]["message"]["content"]

In [ ]:
scenario_list = []

In [ ]:
for i in tqdm(range(NUMBER_OF_SCENARIOS)):

  system_content = scenario_data[i]["system_content"]
  user_content = scenario_data[i]["user_content"]
  scenario_info = scenario_data[i]["scenario_info"]

  try:
    response = send_gigachat_request(system_content, user_content)

    scenario_info["response"] = response
    scenario_list.append(scenario_info)
  except:
    print("Problem!")
    continue
  # time.sleep(0.2)

df = pd.DataFrame(scenario_list)
last_successfull_name = f"eng_response_gigachat.pickle"
df.to_pickle(last_successfull_name)
files.download(last_successfull_name)

  0%|          | 0/1500 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
len(scenario_list)

1273